In [1]:
import os
import sys
import pprint
import pandas as pd
import numpy as np

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)

from GlobalUtils import GlobalUtils

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.ResultFitter import ResultFitter
import json

utils = GlobalUtils()


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [2]:
def clear_nans(result_file_name, num_classes):
    resultProcessor = GridSearchResultProcessor(2)
    df = resultProcessor.get_results_df_by_name(result_file_name, num_classes)
    df_copy = df.copy()
    no_nans = df_copy.dropna()
    clear_results_df(result_file_name, num_classes)
    save_results_df(no_nans, result_file_name, num_classes)

def save_results_df(results_df, file_name, num_classes):
    results_df.to_csv(f"{get_results_file_path()}/{file_name}", mode = 'w', index=False)

def clear_results_df(file_name, num_classes):
    path = get_results_file_path(num_classes)
    file = f"{path}/{file_name}"
    if os.path.isfile(file):
        f = open(file, "w+")
        f.close()

def get_results_file_path(num_classes):
    utils = GlobalUtils()
    file_path = f'{utils.base_dir}/GridSearchResults/{num_classes}_classes'
    return file_path

# Potentially very useful tools for removing duplicates in this function
def delete_n_last_rows(result_file_name, n):
    processor = GridSearchResultProcessor(2)
    df = processor.get_results_df_by_name(result_file_name)
    reduced_df = df.copy()[:-n]
    processor.clear_results_df(result_file_name)
    processor.save_results_df(reduced_df, result_file_name)

def add_f1_score(df):
    df_f1 = df.copy()
    df_f1.columns=df_f1.columns.str.strip()
    all_train_precision = df_f1['train_precision']
    all_train_recall = df_f1['train_recall']
    all_val_precision = df_f1['val_precision']
    all_val_recall = df_f1['val_recall']
    f1_train = create_f1_list(all_train_precision, all_train_recall)
    f1_val = create_f1_list(all_val_precision, all_val_recall)
    df_f1['train_f1'] = f1_train
    df_f1['val_f1'] = f1_val
    return df_f1

def add_fbeta_score(df, beta):
    df_fbeta = df.copy()
    all_train_precision = df_fbeta['train_precision']
    all_train_recall = df_fbeta['train_recall']
    all_val_precision = df_fbeta['val_precision']
    all_val_recall = df_fbeta['val_recall']
    fbeta_train = create_fbeta(all_train_precision, all_train_recall, beta)
    fbeta_val = create_fbeta(all_val_precision, all_val_recall, beta)
    df_fbeta[f'train_f{beta}'] = fbeta_train
    df_fbeta[f'val_f{beta}'] = fbeta_val
    return df_fbeta

def create_fbeta(precision, recall, beta):
    fbeta = np.zeros((len(precision), 1))
    for i in range(len(precision)):
        if precision[i] != np.nan and recall[i] != np.nan:
            fbeta[i] = fbeta_score(precision[i], recall[i], beta)
        else:
            fbeta[i] = np.nan
    return fbeta

def fbeta_score(precision, recall, beta):
    return ((1+beta**2)*precision*recall)/((beta**2)*precision + recall)





In [2]:
processor = GridSearchResultProcessor(2)
df = processor.get_results_df_by_name(result_file_name)
#processor.save_results_df(df, result_file_name)
df

NameError: name 'GridSearchResultProcessor' is not defined

In [20]:
def f1_score(precision, recall):
    f1 = 2*((precision*recall)/(precision + recall))
    return f1

def create_f1_list(precision_df, recall_df):
    f1 = []
    for i in range(len(precision_df)):
        f1.append(f1_score(precision_df.loc[i], recall_df.loc[i]))
    return f1


#result_file_name = 'results_DENSE_noiseNotNoise_bandpass-2.0-4.0_timeAug_normalize_noiseAug_earlyS_numChannels-3.csv'
#result_file_name = 'results_DENSE_grow_noiseNotNoise_timeAug_standard_noiseAug_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3.csv'
#result_file_name = 'results_DENSE_grow_noiseNotNoise_timeAug_normalize_noiseAug_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3_f1.csv'
#result_file_name = 'results_DENSE_grow_noiseNotNoise_timeAug_standard_noiseAug_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3_f3.csv'
#result_file_name = 'results_DENSE_noiseNotNoise_timeAug_robust_noiseAug_earlyS_numChannels-3.csv'
#result_file_name = 'results_DENSE_baseline_noiseNotNoise_timeAug_standard_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3.csv'
#result_file_name = 'results_DENSE_grow_earthExplo_timeAug_standard_noiseAug_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3_f2.csv'
#result_file_name = 'results_InceptionTime_earthExplo_detrend_timeAug_sscale_noiseAug_earlyS_highpass-0.1.csv'
#result_file_name = 'results_InceptionTime_noiseNotNoise_detrend_timeAug_sscale_noiseAug_earlyS_highpass-0.1.csv'
#result_file_name = 'results_InceptionTime_NARROW_noiseNotNoise_detrend_timeAug_sscale_noiseAug_earlyS_highpass-0.1.csv'

# result_file_name = 'results_DENSE_grow_noiseNotNoise_timeAug_standard_noiseAug_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3_f1.csv' Best Accuracy: 74.5%
# result_file_name = 'results_DENSE_grow_noiseNotNoise_timeAug_standard_noiseAug_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3_f3.csv' Best: 74.7
# result_file_name = 'results_DENSE_baseline_noiseNotNoise_timeAug_standard_earlyS_unbalanced_valtest_subsample-0.25_numChannels-3.csv' Best: 86%


gp = GridSearchResultProcessor(2, None, None, None, None, None, None)
#df = gp.get_results_df_by_name(result_file_name)
df = gp.get_results_df_by_name_and_history_folder(result_file_name, "April 18")
#GridSearchResultProcessor(2).clear_nans(result_file_name)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
beta = 2

df
#df_f1 = add_fbeta_score(df, 3)
df_f1 = df
#df_f1

In [22]:
sorted_f1_df = df_f1.sort_values(by=['val_accuracy', 'train_accuracy'], axis = 0, ascending=False)
sorted_f1_df.iloc[0]

units                                              200
output_layer_activation                        sigmoid
optimizer                                      rmsprop
learning_rate                                     0.01
epochs                                              50
dense_activation                                  relu
batch_size                                          64
train_loss                                         NaN
train_accuracy                                0.999905
train_precision                               0.999952
train_recall                                  0.999855
val_loss                                      8.679957
val_accuracy                                  0.864171
val_precision                                 0.898316
val_recall                                    0.831037
confusion_matrix           [[2812  314]\n [ 564 2774]]
Name: 9, dtype: object

Best so far 3n (val_acc):
 - april 19: 68%
 - april 18: 73% (decay) (THIS IS BETTER: 86%)
 - 

 

In [11]:
sorted_f1_df = df_f1.sort_values(by=[f'val_f{beta}', f'train_f{beta}'], axis = 0, ascending=False)
omitted_df = sorted_f1_df[sorted_f1_df['val_recall'] != 1.00]
omitted_df


,meier_load,activation,batch_size,dropout_T_bn_F,dropout_rate,epochs,growth_sequence,l1_r,l2_r,learning_rate,num_layers,optimizer,output_layer_activation,units,use_layerwise_dropout_batchnorm,train_loss,train_accuracy,train_precision,train_recall,train_f2,val_loss,val_accuracy,val_precision,val_recall,val_f2,confusion_matrix
26,False,relu,256,False,0.001,50,"[1, 2]",0.0000,0.0010,0.010,2,sgd,sigmoid,320,True,inf,0.882275,0.856881,0.917050,0.904350,2.018372,0.601262,0.129745,0.740157,0.381339,[[1813 1261]\n [ 66 188]]
20,False,relu,256,False,0.001,50,"[1, 1]",0.0000,0.0001,0.010,2,sgd,sigmoid,320,True,inf,0.830249,0.808483,0.864264,0.852501,0.958013,0.595853,0.124052,0.708661,0.364816,[[1803 1271]\n [ 74 180]]
16,False,relu,256,False,0.001,50,"[1, 2]",0.0000,0.0001,0.010,2,sgd,sigmoid,328,True,inf,0.807861,0.835328,0.765547,0.778555,inf,0.669772,0.136090,0.622047,0.362885,[[2071 1003]\n [ 96 158]]
31,False,relu,256,False,0.001,50,"[1, 2]",0.0001,0.0001,0.010,2,sgd,sigmoid,320,True,inf,0.789795,0.822556,0.737489,0.753065,5.959789,0.659255,0.133333,0.629921,0.361011,[[2034 1040]\n [ 94 160]]
15,False,relu,256,False,0.001,50,"[1, 2]",0.0000,0.0001,0.010,2,sgd,sigmoid,312,True,inf,0.829077,0.836349,0.817060,0.820846,0.905974,0.649339,0.130364,0.633858,0.357619,[[2000 1074]\n [ 93 161]]
22,False,relu,256,False,0.010,50,"[1, 2]",0.0000,0.0001,0.010,2,sgd,sigmoid,320,True,inf,0.848193,0.876884,0.809127,0.821828,1.018023,0.679387,0.136079,0.598425,0.356306,[[2109 965]\n [ 102 152]]
19,False,relu,256,False,0.001,50,"[1, 4]",0.0000,0.0001,0.010,2,sgd,sigmoid,320,True,inf,0.808032,0.750469,0.921310,0.881190,1.131618,0.500000,0.110927,0.791339,0.355375,[[1463 1611]\n [ 53 201]]
21,False,relu,256,False,0.100,50,"[1, 2]",0.0000,0.0001,0.010,2,sgd,sigmoid,320,True,inf,0.820703,0.783663,0.884585,0.862373,inf,0.554087,0.116105,0.732283,0.355233,[[1658 1416]\n [ 68 186]]
2,False,relu,512,False,0.001,25,"[1, 2]",0.0000,0.0001,0.010,2,sgd,sigmoid,320,True,inf,0.734155,0.695935,0.829008,0.798472,inf,0.513346,0.108642,0.775330,0.348101,[[1401 1444]\n [ 51 176]]
25,False,relu,256,False,0.001,50,"[1, 2]",0.0000,0.0100,0.010,2,sgd,sigmoid,320,True,13.847049,0.808521,0.836931,0.765008,0.778387,14.247454,0.662260,0.128840,0.594488,0.345064,[[2053 1021]\n [ 103 151]]


In [10]:
sorted_f1_df = df_f1.sort_values(by=['val_precision', 'train_precision'], axis = 0, ascending=False)
sorted_f1_df

,meier_load,use_layerwise_dropout_batchnorm,units,output_layer_activation,optimizer,num_layers,learning_rate,l2_r,l1_r,growth_sequence,epochs,dropout_rate,dropout_T_bn_F,batch_size,activation,train_loss,train_accuracy,train_precision,train_recall,train_f3,val_loss,val_accuracy,val_precision,val_recall,val_f3,confusion_matrix,train_f1,val_f1
103,False,True,550,sigmoid,sgd,1,0.1000,0.1000,0.3000,[1],50,0.200,True,64,softmax,2.244472e+04,0.507777,0.861111,0.001496,0.001662,2.244450e+04,0.483911,1.000000,0.000599,0.000666,[[3126 0]\n [3336 2]],0.002987,0.001197
57,False,False,620,sigmoid,sgd,1,0.0100,0.1000,0.2000,[1],50,0.001,True,64,relu,inf,0.507349,0.833333,0.000483,0.000536,2.235506e+03,0.483911,1.000000,0.000599,0.000666,[[3126 0]\n [3336 2]],0.000965,0.001197
12,False,True,360,sigmoid,rmsprop,1,0.0001,0.0000,0.2000,[1],50,0.000,True,256,tanh,8.350874e+01,0.507364,0.538462,0.000340,0.000377,8.350562e+01,0.485156,1.000000,0.001515,0.001683,[[3100 0]\n [3295 5]],0.000680,0.003025
60,False,True,450,sigmoid,sgd,3,0.1000,0.0010,0.1000,"[1, 1, 1]",50,0.300,False,256,relu,inf,0.507050,0.588235,0.000483,0.000537,4.441611e+03,0.485156,0.857143,0.001818,0.002020,[[3099 1]\n [3294 6]],0.000965,0.003628
86,False,True,50,sigmoid,adam,5,0.0010,0.0001,0.0000,"[1, 2, 2, 2, 2]",50,0.000,False,64,tanh,4.225689e-01,0.747669,0.797109,0.654635,0.666549,6.985551e-01,0.751083,0.838887,0.641102,0.656583,[[2715 411]\n [1198 2140]],0.718881,0.726779
87,False,True,150,sigmoid,adam,1,0.0010,0.3000,0.2000,[1],50,0.200,False,128,tanh,6.021813e+01,0.510337,0.780876,0.009469,0.010507,6.021293e+01,0.487656,0.838710,0.007879,0.008745,[[3095 5]\n [3274 26]],0.018711,0.015611
90,False,True,490,sigmoid,sgd,4,0.0010,0.3000,0.0000,"[1, 4, 8, 8]",50,0.010,True,256,relu,inf,0.509742,0.692053,0.010097,0.011201,1.686651e+03,0.487969,0.780488,0.009697,0.010760,[[3091 9]\n [3268 32]],0.019904,0.019156
20,False,False,360,sigmoid,sgd,5,0.0100,0.0000,0.0010,"[1, 2, 4, 4, 2]",50,0.200,True,128,relu,1.376427e+02,0.510623,0.707317,0.012609,0.013983,1.376412e+02,0.488594,0.764706,0.011818,0.013109,[[3088 12]\n [3261 39]],0.024776,0.023276
65,False,False,340,sigmoid,sgd,2,0.0010,0.0100,0.0010,"[1, 2]",50,0.300,False,128,relu,inf,0.527796,0.761677,0.061452,0.067674,7.307719e+01,0.500938,0.757282,0.047273,0.052163,[[3050 50]\n [3144 156]],0.113728,0.088991
46,False,False,410,sigmoid,sgd,5,0.0100,0.0100,0.0001,"[1, 2, 4, 4, 2]",50,0.010,True,64,tanh,5.527903e+01,0.509061,0.812500,0.005019,0.005572,5.527962e+01,0.484994,0.736842,0.004194,0.004657,[[3121 5]\n [3324 14]],0.009976,0.008341


In [9]:
sorted_f1_df = df_f1.sort_values(by=['val_loss', 'train_loss'], axis = 0, ascending=True)
sorted_f1_df

,optimizer,num_channels,learning_rate,epochs,batch_size,use_residuals,use_bottleneck,shortcut_activation,reg_shortcut,reg_module,output_activation,num_filters,nr_modules,module_output_activation,module_activation,l2_r,l1_r,kernel_size,bottleneck_size,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
15,adam,3,0.0010,100,128,False,True,relu,False,True,sigmoid,30,9,relu,softmax,0.0000,0.0000,50,26,0.447122,0.796426,0.806589,0.813953,0.546832,0.727362,0.733328,0.729632,0.810254,0.731475
188,sgd,3,0.1000,100,128,False,True,softmax,False,False,sigmoid,26,9,relu,softmax,0.0001,0.0001,30,28,0.506258,0.755922,0.770977,0.788066,0.576602,0.698259,0.711501,0.724869,0.779428,0.718123
266,rmsprop,3,0.0010,100,128,False,True,softmax,False,False,sigmoid,42,3,softmax,sigmoid,0.0010,0.0001,30,34,0.513432,0.778582,0.782161,0.777087,0.581890,0.708954,0.710776,0.695795,0.779615,0.703206
118,rmsprop,3,0.0010,100,256,False,True,tanh,True,False,sigmoid,32,3,softmax,sigmoid,0.1000,0.1000,30,26,0.480957,0.800722,0.816022,0.832273,0.584574,0.696984,0.708171,0.714594,0.824068,0.711368
138,adam,3,0.0100,100,1024,False,True,sigmoid,False,False,sigmoid,24,1,sigmoid,sigmoid,0.1000,0.1000,30,34,0.472440,0.826935,0.788806,0.720368,0.595907,0.723725,0.696126,0.613531,0.753035,0.652224
36,rmsprop,3,0.0010,100,256,False,True,softmax,False,False,sigmoid,42,6,tanh,tanh,0.0000,0.0010,30,28,0.571451,0.728814,0.706183,0.639269,0.595927,0.707540,0.687826,0.617686,0.671062,0.650872
149,rmsprop,3,0.0100,100,256,False,False,relu,False,False,sigmoid,26,3,relu,tanh,0.0000,0.0000,20,28,0.463156,0.768561,0.786901,0.809365,0.596008,0.677783,0.697835,0.729721,0.797975,0.713422
66,rmsprop,3,0.0100,100,128,False,True,relu,False,False,sigmoid,26,9,linear,sigmoid,0.0010,0.1000,40,32,0.542120,0.718574,0.747971,0.799544,0.599841,0.668272,0.695393,0.752135,0.772898,0.722652
55,rmsprop,3,0.0010,100,128,True,False,relu,True,False,sigmoid,42,3,linear,tanh,0.0001,0.0000,40,32,0.487683,0.819297,0.814593,0.798085,0.601262,0.730302,0.723421,0.691196,0.806255,0.706942
319,adam,3,0.0010,100,128,True,False,tanh,True,False,sigmoid,30,1,relu,tanh,0.0001,0.1000,50,30,0.482116,0.772809,0.785739,0.797897,0.605542,0.684236,0.693218,0.695138,0.791771,0.694177


## Findings:
 - New optimal dropout_rate: 0.2 (train_acc: 98%, val_acc: 90%) Old 0.3 (train_acc: 96.8%, val_acc: 90.9%)
 - RELU and Sigmoid activation functions do not work with this model.
 - L1 regularization is counter productive (Less is more) so let l1 = 0.00
 - 32 start neurons appear to be optimal

In [8]:
df.iloc[40:]

,batch_size,epochs,learning_rate,optimizer,bottleneck_size,kernel_size,l1_r,l2_r,module_activation,module_output_activation,nr_modules,num_filters,output_activation,reg_module,reg_shortcut,shortcut_activation,use_bottleneck,use_residuals,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall
40,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,15,38,sigmoid,True,False,relu,True,True,0.248324,0.974392,0.920634,0.862933,0.299261,0.954636,0.896378,0.828245
41,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,17,38,sigmoid,True,False,relu,True,True,0.230950,0.952460,0.927375,0.898652,0.282789,0.932287,0.908168,0.876574
42,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,19,38,sigmoid,True,False,relu,True,True,0.160532,0.990312,0.957003,0.922494,0.312685,0.956051,0.910085,0.856171
43,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,23,38,sigmoid,True,False,relu,True,True,0.283635,0.989134,0.919830,0.847877,0.375227,0.970384,0.894957,0.810736
44,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,25,38,sigmoid,True,False,relu,True,True,0.312877,0.921102,0.922735,0.923586,0.362894,0.889131,0.889063,0.884243
45,128,100,0.0005,adam,28,60,0.0000,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.261194,0.963081,0.946159,0.927156,0.328130,0.930201,0.911009,0.885111
46,128,100,0.0010,adam,28,60,0.1000,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,32.961491,0.918127,0.690320,0.413506,32.966877,0.908482,0.691122,0.412241
47,128,100,0.0010,adam,28,60,0.0010,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,1.153279,0.953837,0.771943,0.568403,1.137331,0.953338,0.778267,0.576472
48,128,100,0.0010,adam,28,60,0.0001,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.482087,0.965123,0.893318,0.814672,0.521788,0.955559,0.883381,0.799595
49,128,100,0.0010,adam,28,30,0.0000,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.769849,0.573269,0.628258,0.985014,0.802668,0.567236,0.623722,0.984517


In [9]:
def f1_score(precision, recall):
    f1 = 2*((precision*recall)/(precision + recall))
    return f1

def create_f1_list(precision_df, recall_df):
    f1 = []
    for i in range(len(precision_df)):
        f1.append(f1_score(precision_df.loc[i], recall_df.loc[i]))
    return f1

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall,train_f1,val_f1
0,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32,0.843425,0.928650,0.862717,0.857514,0.970667,0.868316,0.775568,0.778812,0.891666,0.821132
1,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32,0.856955,0.949212,0.845839,0.809346,0.980064,0.903571,0.766572,0.724409,0.873717,0.804132
2,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.400,17,5,0.0001,0.001,sigmoid,same,32,0.832348,0.944839,0.879387,0.867630,1.003364,0.888981,0.783144,0.768074,0.904590,0.824117
3,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.200,17,5,0.0001,0.001,sigmoid,same,32,0.759288,0.980216,0.920956,0.898200,1.083491,0.900166,0.794034,0.774517,0.937417,0.832628
4,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.100,17,5,0.0001,0.001,sigmoid,same,32,0.765186,0.966016,0.935662,0.935273,1.217005,0.874209,0.786458,0.790981,0.950396,0.830515
5,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.010,17,5,0.0001,0.001,sigmoid,same,32,0.782132,0.983592,0.944435,0.931207,1.338903,0.889803,0.787405,0.774517,0.956683,0.828167
6,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.001,17,5,0.0001,0.001,sigmoid,same,32,0.793265,0.982209,0.929980,0.910209,1.372538,0.886949,0.779356,0.763780,0.944839,0.820769
7,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.000,17,5,0.0001,0.001,sigmoid,same,32,1.044652,0.971432,0.922752,0.909552,1.513449,0.885833,0.776989,0.760916,0.939474,0.818637
8,64,33,0.01000,2,adam,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,2.342889,0.822314,0.720338,0.734196,2.349560,0.814815,0.708807,0.724409,0.775760,0.766957
9,64,33,0.01000,2,rmsprop,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,1.584430,0.763097,0.711021,0.813967,1.595288,0.757818,0.705492,0.815319,0.787711,0.785517


In [10]:
sorted_f1_df

,batch_size,epochs,learning_rate,optimizer,bottleneck_size,kernel_size,l1_r,l2_r,module_activation,module_output_activation,nr_modules,num_filters,output_activation,reg_module,reg_shortcut,shortcut_activation,use_bottleneck,use_residuals,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
36,128,100,0.0010,adam,28,60,0.0000,0.0001,softmax,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.813371,0.988544,0.594751,0.186206,0.848605,0.980887,0.598509,0.185646,0.283617,0.283390
15,64,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.435595,0.988876,0.836395,0.678277,0.493297,0.974793,0.825863,0.662540,0.749083,0.735240
34,128,100,0.0010,adam,28,60,0.0000,0.0001,linear,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.496191,0.975979,0.782449,0.576443,0.512352,0.972373,0.781179,0.570395,0.663831,0.659351
43,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,23,38,sigmoid,True,False,relu,True,True,0.283635,0.989134,0.919830,0.847877,0.375227,0.970384,0.894957,0.810736,0.882389,0.850768
2,128,100,0.0010,adam,28,60,0.0000,0.1000,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.689628,0.963311,0.764990,0.547803,0.686028,0.969460,0.773366,0.555781,0.638431,0.646764
23,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,21,38,sigmoid,False,False,relu,True,True,0.218071,0.984858,0.911397,0.834483,0.271246,0.969129,0.893395,0.808566,0.871246,0.848866
54,128,100,0.0010,adam,28,90,0.0000,0.0001,tanh,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.213274,0.990827,0.933453,0.874095,0.343570,0.966440,0.898082,0.820865,0.902799,0.857739
35,128,100,0.0010,adam,28,60,0.0000,0.0001,relu,sigmoid,21,38,sigmoid,True,False,relu,True,True,0.712026,0.974964,0.569001,0.135964,0.713899,0.964876,0.573082,0.135147,0.219482,0.218715
28,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,21,34,sigmoid,True,False,relu,True,True,0.186433,0.989786,0.945510,0.899588,0.379036,0.957833,0.899361,0.831573,0.921978,0.864139
42,128,100,0.0010,adam,28,60,0.0000,0.0001,tanh,sigmoid,19,38,sigmoid,True,False,relu,True,True,0.160532,0.990312,0.957003,0.922494,0.312685,0.956051,0.910085,0.856171,0.939432,0.882305


In [11]:
pd.to_numeric(df_f1['val_f1'], errors='coerce')

0     0.914104
1     0.892273
2     0.646764
3     0.878245
4     0.917572
5     0.907069
6     0.916099
7     0.896317
8     0.877422
9     0.898971
10    0.921893
11    0.906393
12    0.892303
13    0.753578
14    0.901630
15    0.735240
16    0.894553
17    0.876120
18    0.680396
19    0.659281
20    0.895371
21    0.849455
22    0.908697
23    0.848866
24    0.888820
25    0.857846
26    0.913061
27    0.913520
28    0.864139
29    0.865125
30    0.903461
31    0.913006
32    0.918070
33    0.902414
34    0.659351
35    0.218715
36    0.283390
37    0.897636
38    0.902109
39    0.910168
40    0.860965
41    0.892091
42    0.882305
43    0.850768
44    0.886646
45    0.897873
46    0.516437
47    0.662341
48    0.839402
49    0.763649
50    0.908700
51    0.908270
52    0.879887
53    0.906314
54    0.857739
Name: val_f1, dtype: float64

In [12]:
df_f1['val_f1'].isnull().values.any()

False

In [13]:
df_f1.columns

Index(['batch_size', 'epochs', 'learning_rate', 'optimizer', 'bottleneck_size',
       'kernel_size', 'l1_r', 'l2_r', 'module_activation',
       'module_output_activation', 'nr_modules', 'num_filters',
       'output_activation', 'reg_module', 'reg_shortcut',
       'shortcut_activation', 'use_bottleneck', 'use_residuals', 'train_loss',
       'train_accuracy', 'train_precision', 'train_recall', 'val_loss',
       'val_accuracy', 'val_precision', 'val_recall', 'train_f1', 'val_f1'],
      dtype='object')

In [36]:
import pandas as pd

a =  {'there' : [1],
      'hello': [2]}

df = pd.DataFrame(np.array([[0, 5], [6,6]]), columns=["hello", "there"])

temp = pd.DataFrame.from_dict(a, orient = 'columns')
temp = temp.reindex(df.columns, axis=1)

df = df.append(temp, ignore_index = True)

print(df)


   hello  there
0      0      5
1      6      6
2      2      1


In [25]:
temp

,there,hello
0,1,2


In [15]:
np.concatenate(1, np.arange(3, 40, 3))

TypeError: dispatcher for __array_function__ did not return an iterable

In [9]:
def create_dropout_params(dropout_center):
    min_dropout = 0
    max_dropout = 0.5
    new_dropout = [dropout_center*10**x for x in range(-2, 2, 1)]
    new_dropout.append((dropout_center*10)/2)
    for idx, param in enumerate(new_dropout):
        new_dropout[idx] = max(min(max_dropout, param), min_dropout)
    return list(set(new_dropout))

In [13]:
create_dropout_params(0.01)

[0.1, 0.05, 0.01, 0.0001, 0.001]

In [23]:
def a(*b):
    print(b[0]['hello'])

In [24]:
c = {'hello': "YES",
     'no': "tits"}

In [25]:
a(c)

YES
